## Finales Modell
Nach Grid- und Random-Search werden die Ergebnisse der verschiedenen Modelle miteinander verglichen. Das beste Modell war eines aus der Grid-Search. Die Parameter werden übernommen und in das Modell eingepflegt. Anschließend wirt das Modell erstmalig auf die Testdaten losgelassen

In [1]:
import os
import numpy as np
import shutil
import tensorflow as tf
from typing import Tuple

from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import ParameterGrid
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop

from tf_utils.dogscatsDataAdvanced import DOGSCATS

In [2]:
np.random.seed(0)
tf.random.set_seed(0)

In [3]:
LOGS_DIR = os.path.abspath('C:/Selbststudium/Udemy/Udemy_Tensorflow/logs/Cifar/')
if not os.path.exists(LOGS_DIR):
    os.mkdir(LOGS_DIR)

In [4]:
def build_model(
    img_shape: Tuple[int, int, int],
    num_classes: int,
    optimizer: tf.keras.optimizers.Optimizer,
    learning_rate: float,
    filter_block_1: int,
    kernel_size_block_1: int,
    filter_block_2: int,
    kernel_size_block_2: int,
    filter_block_3: int,
    kernel_size_block_3: int,
    dense_layer_size: int
) -> Model:
    input_img = Input(shape=img_shape)

    x = Conv2D(filters=filter_block_1, kernel_size=kernel_size_block_1, padding='same')(input_img)
    x = Activation('relu')(x)
    x = Conv2D(filters=filter_block_1, kernel_size=kernel_size_block_1, padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Conv2D(filters=filter_block_2, kernel_size=kernel_size_block_2, padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(filters=filter_block_2, kernel_size=kernel_size_block_2, padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Conv2D(filters=filter_block_3, kernel_size=kernel_size_block_3, padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(filters=filter_block_3, kernel_size=kernel_size_block_3, padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D()(x)

    x = Flatten()(x)
    x = Dense(units=dense_layer_size)(x)
    x = Activation('relu')(x)
    x = Dense(units=num_classes)(x)
    y_pred = Activation('softmax')(x)

    # Jetzt muss noch ein Modell Objekt mit eben obiger Struktur erstellt werden
    model = Model(
        inputs = [input_img],
        outputs = [y_pred]
    )
    
    opt = optimizer(learning_rate=learning_rate)
    
    model.compile(
        loss='categorical_crossentropy', # wird bei Kategorie-Problemen mit mehr als 2 Klassen genommen
        optimizer=opt,
        metrics=['accuracy']
    )

    return model

In [5]:
data = DOGSCATS()

train_dataset = data.get_train_set()
val_dataset = data.get_val_set()
test_dataset = data.get_test_set()

img_shape = data.img_shape
num_classes = data.num_classes

epochs = 30
batch_size = 256

# Hier nun die Parameter des Models der GridSearch eintragen
# param_grid={...} wird hier nicht mehr benötigt
# Best model params
optimizer = Adam
learning_rate = 0.001
filter_block1 = 32
kernel_size_block1 = 3
filter_block2 = 64
kernel_size_block2 = 3
filter_block3 = 64
kernel_size_block3 = 7
dense_layer_size = 512



In [6]:
model = build_model(
    img_shape,
    num_classes,
    optimizer,
    learning_rate,
    filter_block1,
    kernel_size_block1,
    filter_block2,
    kernel_size_block2,
    filter_block3,
    kernel_size_block3,
    dense_layer_size,
)

model_log_dir = os.path.join(LOGS_DIR, "modelBest")

tb_callback = TensorBoard(
    log_dir=model_log_dir,
    histogram_freq=0,
    profile_batch=0
)

model.fit(
    train_dataset,
    verbose=1,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[tb_callback],
    validation_data=val_dataset,
)

val_accuracy = model.evaluate(
    test_dataset, 
    batch_size=batch_size,
    verbose=0
)

Epoch 1/30
105/105 [==============================] - 16s 70ms/step - loss: 0.9532 - accuracy: 0.5159 - val_loss: 0.6897 - val_accuracy: 0.5446
Epoch 2/30
105/105 [==============================] - 7s 63ms/step - loss: 0.6676 - accuracy: 0.5877 - val_loss: 0.6593 - val_accuracy: 0.6321
Epoch 3/30
105/105 [==============================] - 7s 64ms/step - loss: 0.6179 - accuracy: 0.6583 - val_loss: 0.6009 - val_accuracy: 0.6655
Epoch 4/30
105/105 [==============================] - 7s 64ms/step - loss: 0.5463 - accuracy: 0.7251 - val_loss: 0.5309 - val_accuracy: 0.7291
Epoch 5/30
105/105 [==============================] - 7s 64ms/step - loss: 0.4978 - accuracy: 0.7622 - val_loss: 0.4784 - val_accuracy: 0.7628
Epoch 6/30
105/105 [==============================] - 7s 64ms/step - loss: 0.4507 - accuracy: 0.7936 - val_loss: 0.4530 - val_accuracy: 0.7876
Epoch 7/30
105/105 [==============================] - 7s 65ms/step - loss: 0.4291 - accuracy: 0.8026 - val_loss: 0.4364 - val_accuracy: 0.795

In [8]:
print(f"Test performance best model: {val_accuracy}")

Test performance best model: [0.9349769949913025, 0.8416833877563477]
